![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [45]:
import pandas as pd
import numpy as np

# Start coding here...

df = pd.read_csv("bank_marketing.csv")

#df.head()

In [46]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [48]:
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


In [49]:
print(client['job'].value_counts())

admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64


In [50]:
client['job'] = client['job'].str.replace(".", "_")
#client['job'].head()

In [51]:
print(client['education'].value_counts())

university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: education, dtype: int64


In [52]:
client['education'] = client['education'].str.replace(".", "_")
client['education'] = client['education'].replace("unknown", np.NaN)
print(client['education'].value_counts())

university_degree      12168
high_school             9515
basic_9y                6045
professional_course     5243
basic_4y                4176
basic_6y                2292
illiterate                18
Name: education, dtype: int64


In [53]:
client['education'].isnull().sum()

1731

In [54]:
print(client['credit_default'].value_counts())

no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64


In [55]:
mapping = {'yes':1, 'no':0, 'unknown':0}
client['credit_default'] = client['credit_default'].replace(mapping)
print(client['credit_default'].value_counts())

0    41185
1        3
Name: credit_default, dtype: int64


In [56]:
client.dtypes

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default     int64
mortgage          object
dtype: object

In [57]:
print(client['mortgage'].value_counts())

yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64


In [58]:
client['mortgage'] = client['mortgage'].replace(mapping)
print(client['mortgage'].value_counts())
client.dtypes

1    21576
0    19612
Name: mortgage, dtype: int64


client_id          int64
age                int64
job               object
marital           object
education         object
credit_default     int64
mortgage           int64
dtype: object

In [59]:
client['credit_default'] = client['credit_default'].astype('bool')
client['mortgage'] = client['mortgage'].astype('bool')
client.dtypes

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default      bool
mortgage            bool
dtype: object

In [60]:
client.to_csv('client.csv', index=False)

In [61]:
campaign = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,261,0,nonexistent,no
1,1,1,149,0,nonexistent,no
2,2,1,226,0,nonexistent,no
3,3,1,151,0,nonexistent,no
4,4,1,307,0,nonexistent,no


In [62]:
print(campaign['previous_outcome'].value_counts())

nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64


In [63]:
mapping2 = {'success':1, 'failure':0, 'nonexistent':0}
campaign['previous_outcome'] = campaign['previous_outcome'].replace(mapping2)

In [64]:
campaign['previous_outcome'] = campaign['previous_outcome'].astype('bool')
print(campaign['previous_outcome'].value_counts())

False    39815
True      1373
Name: previous_outcome, dtype: int64


In [65]:
print(campaign['campaign_outcome'].value_counts())

no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [66]:
mapping3 = {'yes':1, 'no':0}
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace(mapping3)
campaign['campaign_outcome'] = campaign['campaign_outcome'].astype('bool')
print(campaign['campaign_outcome'].value_counts())

False    36548
True      4640
Name: campaign_outcome, dtype: int64


In [67]:
import datetime

In [68]:
date_column = df[['day', 'month']]
date_column.head()

,day,month
0,13,may
1,19,may
2,23,may
3,27,may
4,3,may


In [69]:
date_column['day'] = date_column['day'].astype('str')
#date['month'] = date['month'].astype('date')
#date.dtypes

In [70]:
#date_column['day'] = pd.to_datetime(date_column['day'])
#date_column['month'] = pd.to_datetime(date_column['month'])

In [71]:
date_column.dtypes

day      object
month    object
dtype: object

In [72]:
date_column['test_date'] = "2022" + "-" + date_column['month'] + "-" + date_column['day']

In [73]:
date_column['test_date'].head()

0    2022-may-13
1    2022-may-19
2    2022-may-23
3    2022-may-27
4     2022-may-3
Name: test_date, dtype: object

In [74]:
date_column['test_date'] = pd.to_datetime(date_column['test_date'])

In [75]:
date_column.dtypes

day                  object
month                object
test_date    datetime64[ns]
dtype: object

In [76]:
#x =  datetime.datetime(2022, date['month'], date['day']).strftime("%Y-%m-%m")

date_column['last_contact_date'] = date_column['test_date'].dt.strftime("%Y-%m-%d")
date_column.head()

,day,month,test_date,last_contact_date
0,13,may,2022-05-13,2022-05-13
1,19,may,2022-05-19,2022-05-19
2,23,may,2022-05-23,2022-05-23
3,27,may,2022-05-27,2022-05-27
4,3,may,2022-05-03,2022-05-03


In [77]:
campaign['last_contact_date'] = date_column['last_contact_date']

In [78]:
#campaign.head()
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'])
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB


In [79]:
campaign.to_csv('campaign.csv', index=False)

In [80]:
economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']]
economics.to_csv('economics.csv', index=False)